In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser


In [ ]:
from utils import draw_graph

In [ ]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
template = """
You are helpfull assistant your task is to give answer of the user questions. Do not give lengthy answers.
If a question includes harmful, offensive, or inappropriate language, respond politely by redirecting to a more suitable topic or offering support in a positive, constructive manner.
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2:1b")

# chain = prompt | model

In [ ]:
tool = DuckDuckGoSearchRun(output_format="string")
tools = [tool]

In [ ]:
model_with_tools = model.with_config(tools=tools)

In [ ]:
chain = prompt | model_with_tools | StrOutputParser()

In [ ]:
def chatbot(state: State):
    return {"messages": [chain.invoke(state["messages"])]}

In [ ]:
graph_builder.add_node("chatbot", chatbot)

In [ ]:
graph_builder.add_edge(START, "chatbot")

In [ ]:
graph_builder.add_edge("chatbot", END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
draw_graph(graph)

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][0])


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except Exception as e:
        print(e)